In [114]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
%matplotlib inline
folder = 0

In [115]:
# Video retrieval and frame segmentation, returns the list of frames
def retrieve_and_segment(path):
    list=[]
    frames_full=[]
    vid = cv2.VideoCapture(path)
    fps = int(np.ceil(vid.get(cv2.CAP_PROP_FPS)))
    if not vid.isOpened():
        print("NO VIDEO")
        return list
    i=0
    while True:
        ret,frame = vid.read()
#         display the video (doesn't work in google collab)
#         cv2.imshow('ff',frame)       
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        if not ret:
            break
        if i%fps == 0:
#             HUE range is [0,255]
            frames_full.append(frame)
            frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV_FULL)
            list.append(frame[:,:,0])
        i+=1
    vid.release()
    return list,frames_full

In [116]:
# Convert the HUE channel to histogram, returns the list of histograms
def get_hist(frames,dims):
    hists = []
    shape = frames[0].shape
    l = shape[0]
    w = shape[1]
    for i,frame in enumerate(frames):
        fr=cv2.calcHist([frame],channels=[0],mask=None,histSize=[dims],ranges=[0,256])
        fr/=l*w
        hists.append(fr)
    return np.array(hists)

In [117]:
# #of clusters required
def num_of_clusters(hists):
    k=1
    i=1
    for i in range(len(hists)):
        val = np.linalg.norm(hists[i]-hists[i-1],axis = 0)
        if val>0.25:
            k+=1
    return k

In [118]:
# initialize the clusters with item indices, returns initialized clusters
def initialize_clusters(l,k):
    clusters = np.zeros((l,1))
    per_cluster = np.ceil(l/k)
    xtra = l%k
    added = 0
    cur_cluster = 0
    for i in range(l):
        clusters[i][0] = cur_cluster
        added += 1
        if added == per_cluster:
            added = 0
            cur_cluster += 1
            if cur_cluster == xtra:
                per_cluster -= 1
    return clusters

In [119]:
# calculates the new cluster centers and then assigns frames to clusters
def k_mean_util(items,clusters,old_centers):
    k = len(old_centers)  
    frequency = np.zeros((k,1))
    new_centers = np.zeros((k,items[0].shape[0],1))
    # update centers #1
    for i in range(len(clusters)):
        new_centers[int(clusters[i][0])] += items[i]
        frequency[int(clusters[i][0])] += 1
    # update centers #2
    for i in range(k):
        if frequency[i][0] == 0:
            new_centers[i] = old_centers[i]
        else:
            new_centers[i] /= frequency[i][0]
    # update clusters
    for i in range(len(clusters)):
        euclidean = np.linalg.norm(new_centers - items[i] , axis = 1)
        assigned_cluster = np.argmin(euclidean , axis = 0)
        clusters[i][0] = assigned_cluster
    return new_centers

In [120]:
# k_mean_clustering, returns cluster_centers,clusters
def k_mean_clustering(items,k):
    cluster_centers = np.zeros((k,items[0].shape[0],1))
    clusters = initialize_clusters(len(items),k)
    no_iterations = 20
    i = 1
    while i < no_iterations:
        cluster_centers = k_mean_util(items,clusters,cluster_centers)
        i += 1
    return (cluster_centers,clusters)

In [121]:
# select keyframes from each cluster
def select_keyframes(items,cluster_centers,clusters):
    k = len(cluster_centers)
    keyframes = []
    each_cluster = [[] for i in range(k)]
    for i in range(len(items)):
        each_cluster[int(clusters[i][0])].append(i)
    for i in range(k):
        if len(each_cluster[i]) > 0:
            cluster_hists = [items[j] for j in each_cluster[i]]
            euclidean = np.linalg.norm(cluster_hists - cluster_centers[i] , axis = 1)
            keyframe_index = np.argmin(euclidean)
            keyframes.append(each_cluster[i][keyframe_index])
    keyframes.sort()
    return keyframes

In [122]:
def display_summary(keyframes,frames_full):
    i =0 
    while True:
        frame = frames_full[keyframes[i]]
        cv2.imshow('frame',frame)
        if cv2.waitKey(1000) & 0xFF == ord('q'):
            break;
        i = (i+1)%len(keyframes)
    cv2.destroyAllWindows()

In [123]:
def save_summary(keyframes,frames_full,path):
    l,w,c = frames_full[0].shape
    global folder
    os.mkdir("summaries/%d" % folder)
#     writer = cv2.VideoWriter("summ.avi",cv2.VideoWriter_fourcc(*"XVID"),1,(640,480))
    for i in range(len(keyframes)):
        frame = frames_full[keyframes[i]]
        cv2.imwrite("summaries/{}/frame{}.jpg".format(folder,keyframes[i]) ,frame)
    folder += 1
#         writer.write(frame)
#     writer.release()

In [124]:
def model(file):
#     frames,frames_full = retrieve_and_segment(file)
    hists = get_hist(frames,16)
    k = num_of_clusters(hists)
    cluster_centers,clusters = k_mean_clustering(hists,k)
    keyframes = select_keyframes(hists,cluster_centers,clusters)
#     save_summary(keyframes,frames_full,file)
    display_summary(keyframes,frames_full)

In [125]:
path="Pip - A Short Animated Film.mp4"
frames,frames_full = retrieve_and_segment(path)

In [128]:
model(path)